<a href="https://colab.research.google.com/github/ishandas387/Language-Model-For-Odia/blob/master/WebScaping_odia_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting Headline from Sambad.in

References from iNltk library.

In [ ]:
#importing the required libs
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [ ]:
def bs(resp):

  soup = BeautifulSoup(resp.text, 'html.parser')
  headings =[]
  articles = soup.find_all('article')
  for article in articles:
    headline = str(article.find('h2').text).replace("\n","")
    headline = headline.replace('"','').replace("'","")
    #print(headline)
    headings.append(headline)
  return headings

 

In [ ]:
url = 'http://sambad.in/india-and-beyond/'
res = requests.get(url, headers={'User-Agent': 'Chrome/71.0.3578.98'})
in_headings = bs(res)
df = pd.DataFrame({"headlines" : international_headings})
print(df.shape)
df.to_csv('firstrun.csv',index=False)

(14, 1)


State news - 
This is page wise in a loop


In [ ]:
url = 'http://sambad.in/state/page/{}/'
headings =[]
for i in range(1,900):
  print("reading for page {}".format(i))
  url = url.format(i)
  res = requests.get(url, headers={'User-Agent': 'Chrome/71.0.3578.98'})
  headings.extend(bs(res))
df = pd.DataFrame({"headlines" : headings,"type":"state"})
print(df.shape)
df.to_csv('state_headings_new.csv',index=False)  

reading for page 1
reading for page 2
reading for page 3
reading for page 4
reading for page 5
reading for page 6
reading for page 7
reading for page 8
reading for page 9
reading for page 10
reading for page 11
reading for page 12
reading for page 13
reading for page 14
reading for page 15
reading for page 16
reading for page 17
reading for page 18
reading for page 19
reading for page 20
reading for page 21
reading for page 22
reading for page 23
reading for page 24
reading for page 25
reading for page 26
reading for page 27
reading for page 28
reading for page 29
reading for page 30
reading for page 31
reading for page 32
reading for page 33
reading for page 34
reading for page 35
reading for page 36
reading for page 37
reading for page 38
reading for page 39
reading for page 40
reading for page 41
reading for page 42
reading for page 43
reading for page 44
reading for page 45
reading for page 46
reading for page 47
reading for page 48
reading for page 49
reading for page 50
reading f

Seems fine. The sambad state news has around 4K pages through.


Let me try for another website, which doesnt have page system, rather load more option in ajax call




In [ ]:
df_sports = pd.read_csv("sport_headings.csv")

In [ ]:
df_sports.head()

,headlines,type
0,"କୋହଲି ଗଣିତରେ ଦୁର୍ବଳ, ଧୋନି ସବଳ",sports
1,ଧୋନି ପାରିବା ପର୍ଯ୍ୟନ୍ତ ଖେଳିପାରିବେ: ହସି,sports
2,ଡି’ଭିଲିୟର୍ସଙ୍କ ଆଇପିଏଲ୍‌ ଏକାଦଶରେ ଧୋନି ଅଧିନାୟକ,sports
3,୧୯୯୯ ବିଶ୍ବକପ୍‌ର ମଜାଦାର କାହାଣୀ : ହୋଟେଲ ଆଲ୍‌ମି...,sports
4,ଏକବିଂଶ ଶତାବ୍ଦୀରେ ଜାଦେଜା ଭାରତର ସବୁଠୁ ମୂଲ୍ୟବାନ ...,sports


In [ ]:
df_sports.shape

(13485, 2)

In [ ]:
df_business = pd.read_csv("business_headings.csv")

In [ ]:
df_business.shape

(12586, 2)

In [ ]:
df_combined = df_sports.append(df_business)
df_combined.shape


(26071, 2)

In [ ]:
df_state = pd.read_csv("state_headings_new.csv")
df_combined = df_combined.append(df_state)
df_combined.shape

(37758, 2)

In [ ]:
df_combined.isnull().sum()
df_combined.tail()

,headlines,type
11682,ଉଚ୍ଚଶିକ୍ଷା ବିଭାଗର ଗାଇଡଲାଇନ ଜାରି: ୧୩ରୁ ଅନ୍‌ଲାଇ...,state
11683,ଓଡ଼ିଶାରେ ପ୍ଲାଜମା ଥେରାପି ମାଧ୍ୟମରେ ହେବ କରୋନା ଚିକ...,state
11684,"ଶ୍ରୀକ୍ଷେତ୍ରରେ ସୁନାବେଶରେ ରଥାରୂଢ଼ ଶ୍ରୀଜିଉ, ଅପରୂପ...",state
11685,"କରୋନା ସଂକ୍ରମଣରୁ ସୁସ୍ଥ ହେଲେ ୧୪୯ ଜଣ, ରାଜ୍ୟରେ ସୁ...",state
11686,ରାଜ୍ୟରେ ବଜ୍ରପାତ ସହ ବର୍ଷା ସମ୍ଭାବନା; ୧୬ ଜିଲ୍ଲାକ...,state


In [ ]:
df_combined.to_csv('odia_dataset_sambad_headlines.csv',index=False)  

In [ ]:

print('In Validation data:')
for cls in df_combined['type'].unique():
    print(cls, (df_combined['type'] == cls).sum())

In Validation data:
sports 13485
business 12586
state 11687


Doing it in a diff way


In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 2.8MB/s 


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import re
import pickle
import pandas as pd

In [ ]:
all_sports_headings = []

In [ ]:
for i in range(426,900):
    print("page {}".format(str(i)))
    url = 'http://sambad.in/sports/page/' + str(i) + '/'
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html_doc = ''
    with urlopen(req) as response:
        for line in response:
            line = line.decode('utf-8')
            html_doc = html_doc + line.replace('\n','')
    soup = BeautifulSoup(html_doc, 'html.parser')
    articles = soup.find_all('article')
    for article in articles:
        all_sports_headings.append(article.find('h2').text)
    if i%100 == 0:
        print(len(set(all_sports_headings)))

page 426
page 427
page 428
page 429
page 430
page 431
page 432
page 433
page 434
page 435
page 436
page 437
page 438
page 439
page 440
page 441
page 442
page 443
page 444
page 445
page 446
page 447
page 448
page 449
page 450
page 451
page 452
page 453
page 454
page 455
page 456
page 457
page 458
page 459
page 460
page 461
page 462
page 463
page 464
page 465
page 466
page 467
page 468
page 469
page 470
page 471
page 472
page 473
page 474
page 475
page 476
page 477
page 478
page 479
page 480
page 481
page 482
page 483
page 484
page 485
page 486
page 487
page 488
page 489
page 490
page 491
page 492
page 493
page 494
page 495
page 496
page 497
page 498
page 499
page 500
4995
page 501
page 502
page 503
page 504
page 505
page 506
page 507
page 508
page 509
page 510
page 511
page 512
page 513
page 514
page 515
page 516
page 517
page 518
page 519
page 520
page 521
page 522
page 523
page 524
page 525
page 526
page 527
page 528
page 529
page 530
page 531
page 532
page 533
page 534
page 535
page 

In [ ]:
all_sports_headings = list(set(all_sports_headings))

print(len(all_sports_headings))



8969


In [ ]:
all_business_headings = []

In [ ]:
for i in range(684,900):
    print("page {}".format(str(i)))
    url = 'http://sambad.in/business/page/' + str(i) + '/'
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html_doc = ''
    with urlopen(req) as response:
        for line in response:
            line = line.decode('utf-8')
            html_doc = html_doc + line.replace('\n','')
    soup = BeautifulSoup(html_doc, 'html.parser')
    articles = soup.find_all('article')
    for article in articles:
        all_business_headings.append(article.find('h2').text)
    if i%100 == 0:
        print(len(set(all_business_headings)))

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29
page 30
page 31
page 32
page 33
page 34
page 35
page 36
page 37
page 38
page 39
page 40
page 41
page 42
page 43
page 44
page 45
page 46
page 47
page 48
page 49
page 50
page 51
page 52
page 53
page 54
page 55
page 56
page 57
page 58
page 59
page 60
page 61
page 62
page 63
page 64
page 65
page 66
page 67
page 68
page 69
page 70
page 71
page 72
page 73
page 74
page 75
page 76
page 77
page 78
page 79
page 80
page 81
page 82
page 83
page 84
page 85
page 86
page 87
page 88
page 89
page 90
page 91
page 92
page 93
page 94
page 95
page 96
page 97
page 98
page 99
page 100
1005
page 101
page 102
page 103
page 104
page 105
page 106
page 107
page 108
page 109
page 110
page 111
page 112
page 113
page 114
page 115
page 116
page 117
page 118
page 119
page 120
page 121
page 122
page 

ConnectionResetError: ignored

In [ ]:
#all_business_headings = list(set(all_business_headings))

print(len(all_business_headings))
all_state_headings = []

9548


In [ ]:
for i in range(620,900):
    print("page {}".format(str(i)))
    url = 'http://sambad.in/state/page/' + str(i) + '/'
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html_doc = ''
    with urlopen(req) as response:
        for line in response:
            line = line.decode('utf-8')
            html_doc = html_doc + line.replace('\n','')
    soup = BeautifulSoup(html_doc, 'html.parser')
    articles = soup.find_all('article')
    for article in articles:
        all_state_headings.append(article.find('h2').text)
    if i%100 == 0:
        print(len(set(all_state_headings)))

page 620
page 621
page 622
page 623
page 624
page 625
page 626
page 627
page 628
page 629
page 630
page 631
page 632
page 633
page 634
page 635
page 636
page 637
page 638
page 639
page 640
page 641
page 642
page 643
page 644
page 645
page 646
page 647
page 648
page 649
page 650
page 651
page 652
page 653
page 654
page 655
page 656
page 657
page 658
page 659
page 660
page 661
page 662
page 663
page 664
page 665
page 666
page 667
page 668
page 669
page 670
page 671
page 672
page 673
page 674
page 675
page 676
page 677
page 678
page 679
page 680
page 681
page 682
page 683
page 684
page 685
page 686
page 687
page 688
page 689
page 690
page 691
page 692
page 693
page 694
page 695
page 696
page 697
page 698
page 699
page 700
6992
page 701
page 702
page 703
page 704
page 705
page 706
page 707
page 708
page 709
page 710
page 711
page 712
page 713
page 714
page 715
page 716
page 717
page 718
page 719
page 720
page 721
page 722
page 723
page 724
page 725
page 726
page 727
page 728
page 729
page 

In [ ]:
print(len(all_state_headings))


8980


In [ ]:
all_business_headings = list(set(all_business_headings))
all_sports_headings = list(set(all_sports_headings))
all_state_headings = list(set(all_state_headings))

In [ ]:
print(len(all_business_headings))
print(len(all_state_headings))
print(len(all_sports_headings))

6792
8980
8969


In [ ]:
all_headings = all_state_headings + all_sports_headings + all_business_headings
print(len(all_headings))

24741


In [ ]:

all_labels = ['state']*len(all_state_headings)+ ['sports']*len(all_sports_headings) + ['business']*len(all_business_headings) 

In [ ]:
df = pd.DataFrame({'headings':all_headings, 'label':all_labels})

In [ ]:
df.shape

(24741, 2)

In [ ]:
df.tail()

,headings,label
24736,ଦିନକରେ ନିବେଶକ ହରାଇଲେ ୧.୭ ଲକ୍ଷ କୋଟି,business
24737,ସୋବିସ୍କୋ ବିସ୍କୁଟ ବିକ୍ରିରେ ୬୦% ବୃଦ୍ଧି,business
24738,ରାହୁଲଙ୍କ ସହ ଆଲୋଚନା ବେଳେ ରାଜନଙ୍କ ପରାମର୍ଶ: ଗରି...,business
24739,୨୦୨୦ ଏପ୍ରିଲ୍‌ରୁ ‘ବିଏସ୍‌-୪ କାର୍‌’ ବିକ୍ରି ବନ୍ଦ,business
24740,ଆମାଜନ୍‌ ପେ ଏବେ ଡୋମିନୋସ୍‌ରେ ଗ୍ରହଣୀୟ ହେବ,business


In [ ]:
df.to_csv('odia_news_sambad_headlines.csv',index=False)  